In [1]:
#Módulos utilizados
import pandas as pd
from datetime import date
import csv
from bs4 import BeautifulSoup
import requests
import time

In [2]:
#Utilizando o Selenium como alternativa - método get está sendo bloqueado pelo servidor
#Importações selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

#Importações Selenium Wait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

servico = Service(ChromeDriverManager(version="114.0.5735.90").install())
#servico = Service()
options = webdriver.ChromeOptions()
navegador = webdriver.Chrome(service=servico, options=options)

[WDM] - ====== WebDriver manager ======
[WDM] - There is no [win32] chromedriver for browser  in cache
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\pteix\.wdm\drivers\chromedriver\win32\114.0.5735.90]


In [2]:
#criando as variáveis do dia de hoje
todays_date = date.today()

todays_year = todays_date.year
todays_year_str = str(todays_year)

todays_month = todays_date.month

In [3]:
#Função que obtém um dataframe de terceirizados de uma página html passada como argumento
def obtem_data_frame_terceirizados(site):
    #Entrar no [site] que contém a tabela de terceirizados
    #Carregar a tabela de terceirizados do site para um dataframe
    
    #Carrega uma lista de dataframes. Cada dataframe desta lista é uma tabela do site
    dfs_terceirizados = pd.read_html(site)
    
    #Obter somente o dataframe de terceirizados
    df_terceirizados = dfs_terceirizados[0]
    
    return df_terceirizados


def conta_quantidade_terceirizados(dataframe):
    #Contar e retornar a quantidade de linhas de um [dataframe] de terceirizados
    return len(dataframe.index)

def filtra_dataframe(dataframe: 'dataframe', nome_col: 'str', lista_valores: 'list'):
    #Filtrar um [dataframe] no [campo 'empresa'], com o valor da [empresa escolhida], aplicando
    df_filtrado = dataframe.loc[dataframe[nome_col].isin(lista_valores)]
    return df_filtrado
    
    



In [4]:
#Variáveis de acesso
arquivo_atual = r'C:\Users\pteix\STJ- Superior Tribunal de Justiça\AGS Teams - Documentos\1. Drive H AGS\Assessoria\0 Dados\PLS\GRUPOEXEC\1. TEMAS\GE Rec Tecnol\04. Impressão\QtdTerceirizados.csv'

In [5]:
#Carregando dados do arquivo atual
df_atual = pd.read_csv(arquivo_atual)

#Converte a coluna de datas para o formato de data do pandas
df_atual['data'] = pd.to_datetime(df_atual['data'])

#display(df_atual)
#print(df_atual.info())

In [6]:
#Obtendo a data mais recente
ultima_data = df_atual['data'].max()
ultimo_mes = ultima_data.month

#Calculando data de corte para execução do código
#data_corte = ultima_data - pd.Timedelta(days=180)
data_corte = pd.Timestamp(year = ultima_data.year, month = ultimo_mes + 1, day=1)
ano_corte = data_corte.year
mes_corte = data_corte.month
data_corte = pd.Timestamp(year = ano_corte, month = mes_corte, day=1 )

print(ultima_data)
print(data_corte)

2023-05-01 00:00:00
2023-06-01 00:00:00


In [7]:
html = 'terceirizados_2023_06.html'

In [11]:
#Define cabeçalho para a requisição http
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7'
}

#Inicia dicionário de dataframes
print('iniciando o dicionario')
dfs_terceirizados = {}

#Loop do ano: do ano de corte até o ano atual
print('definindo variável i do loop - ano')
for i in range(ano_corte,todays_year + 1):

    # Define a variável string do ano
    print('definindo variável i_str')
    i_str = str(i)
    print(i_str)
    
    
    #Loop do mês: do mês 1 ao mês 12
    print('Entrando no looping do mês - variável j')
    for j in range(1,13):

        #Constrói a data da iteração
        data_iteracao = pd.Timestamp(year = i, month = j, day= 1)
        
        #Se data da iteração for menor que a data de corte, passa para o próximo valor do iterador de mês
        if data_iteracao < data_corte:
            continue
        
        #testa se o mês da iteração ainda não ocorreu
        print('fazendo o teste do valor da variável j')
        if ((i == todays_year) and (j > mes_corte)):
            break #sai do loop da variável j, responsável pelo mês
        else:
            print('não se trata do mês atual. Definindo variável j_str')
            #Define a variável string do mês da iteração
            print('testando se j é menor que 10')
            if j<10:
                j_str = '0'+str(j)
            else:
                j_str = str(j)
        
        print('definindo a variável string do codigo html')
        # Define a variável string do codigo html
        html_str = i_str + j_str + '01.html'
        print(html_str)
        
        
        #Define a string chave do dicionário
        print('definindo a chave do dicionário')
        dic_chave = data_iteracao

        #Define a variavel da pagina html a ser lida
        print('define a variável da endereco a ser acessado')
        endereco = r'https://www.stj.jus.br/docs_internet/transparencia/stjtransparencia_anexoVI_' + html_str
        
        
        #Inicia uma sessão de acesso ao site:
        #with requests.Session() as s:
        
        
        #    #Verificando disponibilidade da página html
        #    print('tentando obter resposta do servidor')
        #    response = s.get(endereco, headers=headers)
        #    print(response.status_code)
        
        #    #Imprime informações da iteração:
        #print('Iteração {}-{}'.format(i_str, j_str))
        #print('Link: {}'.format(endereco))
        #    #print(response)
        #html = response.text
        
        
        #Navega até o site
        navegador.get(endereco)
        time.sleep(3)
        html = navegador.page_source

        
        #soup = BeautifulSoup(html, 'html.parser')
        #table = soup.find('table')

        #print(soup)
        #print(table)
        
        df = pd.read_html(html)[0]
        dfs_terceirizados[dic_chave] = df
        
        display(df)

        

iniciando o dicionario
definindo variável i do loop - ano
definindo variável i_str
2023
Entrando no looping do mês - variável j
fazendo o teste do valor da variável j
não se trata do mês atual. Definindo variável j_str
testando se j é menor que 10
definindo a variável string do codigo html
20230601.html
definindo a chave do dicionário
define a variável da endereco a ser acessado


ValueError: No tables found

In [ ]:
#Constrói um dicionário de informações extraídas das tabelas de terceirizados
df_informacoes_terceirizados = {}

#Compila valores do dicionário de dataframes obtido
for item in dfs_terceirizados:
    print(item)
    display(dfs_terceirizados[item])
    df_unidades_limpeza = filtra_dataframe(dfs_terceirizados[item], 4 , ["Seção de Limpeza e Conservação","Coordenadoria de Conservação Predial","Seção de Administração de Edifícios"])
    df_terceirizados_limpeza = filtra_dataframe(df_unidades_limpeza, 3, ["Jauzeiro", "Servente", "Supervisor", "Supervisor Geral de Serventes"])
    display(df_terceirizados_limpeza)
    terceirizados_limpeza = conta_quantidade_terceirizados(df_terceirizados_limpeza)
    terceirizados_total = conta_quantidade_terceirizados(dfs_terceirizados[item]) - 1
    
    df_informacoes_terceirizados[item] = (terceirizados_total, terceirizados_limpeza)
    

In [ ]:
#Obtém lista de datas a serem atualizadas
datas_update = [data for data in df_informacoes_terceirizados if len(df_atual.loc[df_atual['data'] == data]) > 0]

#Obtém lista de datas a serem acrescentadas
datas_add = [data for data in df_informacoes_terceirizados if len(df_atual.loc[df_atual['data'] == data]) == 0]

#Atualiza os valores do dataframe com os valores retornados pela consulta atual
for data in datas_update:
    df_atual.loc[df_atual['data'] == data] = [data, df_informacoes_terceirizados[data][0], df_informacoes_terceirizados[data][1]]

#Acrescenta os novos valores obtidos pela consulta no dataframe
for data in datas_add:
    new_row = {'data': [data], 'terceirizados': [df_informacoes_terceirizados[data][0]], 'terceirizados_limpeza': [df_informacoes_terceirizados[data][1]]}
    df_atual = pd.concat([df_atual, pd.DataFrame(new_row)], ignore_index = True)

for data in datas_update:
    display(df_atual.loc[df_atual['data'] == data])

    
for data in datas_add:
    display(df_atual.loc[df_atual['data'] == data])
    
    

    
    

In [ ]:
#Dataframe final
display(df_atual)

#Exporta o dataframe final
#nome_arquivo = r'QtdTerceirizados.csv'
caminho = r'C:\Users\pteix\STJ- Superior Tribunal de Justiça\AGS Teams - Documentos\1. Drive H AGS\Assessoria\0 Dados\PLS\GRUPOEXEC\1. TEMAS\GE Rec Tecnol\04. Impressão\QtdTerceirizados.csv'
df_atual.to_csv(caminho, index = False)